<h2>Asymmetric ciphers</h2>
Senders and receivers use different keys. Senders encrypt with public keys (non-secret) whereas receivers decrypt with private keys (secret). Asymmetric ciphers are typically very slow and can process only very small payloads. Example: PKCS#1 OAEP (RSA).

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

message = b'hello world.'
# 导入RSA公钥
pub_key = RSA.import_key(open('data/rsa_pub_key.pem').read())

# 创建PKCS1OAEP_Cipher，PKCS#1 OAEP is an asymmetric cipher based on RSA and the OAEP padding. 
cipher = PKCS1_OAEP.new(pub_key)
cipher_text = cipher.encrypt(message)
print(cipher_text)

In [ ]:
# 导入RSA私钥
key = RSA.import_key(open('data/rsa_key.pem').read())

# 创建PKCS1OAEP_Cipher，加密时使用公钥，解密时使用私钥
cipher = PKCS1_OAEP.new(key)
plain_text = cipher.decrypt(cipher_text)
print(plain_text)

<h2>Digital signatures</h2>
Digital signatures are based on public key cryptography: the party that signs a message holds the private key, the one that verifies the signature holds the public key，used to guarantee integrity and non-repudiation.

In [ ]:
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto import Hash

message = b'hello world.'
# 导入RSA私钥
key = RSA.import_key(open('data/rsa_key.pem').read())

# 创建hash对象，可用 update() 添加字节对象
hash_obj = Hash.SHA256.new(message)

# 创建signer对象，调用sign()方法并以hash对象为输入参数，返回signature of message
signer = pss.new(key)
signature = signer.sign(hash_obj)
print(signature)

In [ ]:
# 导入RSA公钥
pub_key = RSA.import_key(open('data/rsa_pub_key.pem').read())

# 创建hash对象，可用 update() 添加字节对象
hash_obj = Hash.SHA256.new(message)

# 创建verifier对象，调用sign()方法并以hash对象为输入参数，返回signature of message
verifier = pss.new(pub_key)

try:
    # 调用verifier的 verify() 方法，并以hash对象和signature为输入参数，返回 
    Invoke the verify() method on the verifier, with the hash object and the incoming signature as parameters. If the message is not authentic, an ValueError is raised.
    verifier.verify(hash_obj, signature)
    print('Verify signature: success.')
except ValueError:
    print('Verify signature: fail.')

<h2>Symmetric ciphers</h2>
All parties use the same key, for both decrypting and encrypting data. Symmetric ciphers are typically very fast and can process very large amount of data.<br>
AES (Advanced Encryption Standard) is a symmetric block cipher standardized by NIST . It has a fixed data block size of 16 bytes. Its keys can be 128, 192, or 256 bits long.It is very fast and secure, and it is the de facto standard for symmetric encryption.

In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

message = b'hello world.'

# returns a random byte string of the desired size.
key = get_random_bytes(16)

# 创建CBC加密算法对象
# CBC Mode:Ciphertext Block Chaining,it is a mode of operation where each plaintext block gets XOR-ed with the previous ciphertext block prior to encryption.
cipher = AES.new(key, AES.MODE_CBC)

# A CBC cipher object has a read-only attribute iv, holding the Initialization Vector (bytes). If not present, the library creates a random IV value.
iv = cipher.iv
print(iv)

# The method encrypt() (and likewise decrypt()) of a CBC cipher object expects data to have length multiple of the block size (e.g. 16 bytes for AES). You might need to use Crypto.Util.Padding to align the plaintext to the right boundary.
cipher_text = cipher.encrypt(pad(message, AES.block_size))
print(cipher_text)

In [ ]:
cipher = AES.new(key, AES.MODE_CBC, iv)

plain_text = unpad(cipher.decrypt(cipher_text), AES.block_size)
print(plain_text)

<h2>Hybrid ciphers</h2>
The two types of ciphers above can be combined in a construction that inherits the benefits of both. An asymmetric cipher is used to protect a short-lived symmetric key, and a symmetric cipher (under that key) encrypts the actual message.

In [44]:
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad

message = b'hello world.'

# 导入RSA公钥
pub_key = RSA.import_key(open('data/rsa_pub_key.pem').read())

# returns a random byte string of the desired size.
aes_key = get_random_bytes(16)
print('AES KEY: {}'.format(aes_key))

# Encrypt the AES key with the public RSA key
cipher_rsa = PKCS1_OAEP.new(pub_key)
enc_aes_key = cipher_rsa.encrypt(aes_key)
print('Encrypted AES Key: {}'.format(enc_aes_key))

# Encrypt the message with the AES key
cipher_aes = AES.new(aes_key, AES.MODE_CBC)
cipher_text = cipher_aes.encrypt(pad(message, AES.block_size))
print('Cipher Text: {}'.format(cipher_text))

aes_iv = cipher_aes.iv
print('AES_IV：{}'.format(aes_iv))

AES KEY: b'A[\x9d\xd7?\x11\x93\xa1\xbb\x86\xa0\x16\xc4\x1c\xf1x'
Encrypted AES Key: b'W)k\xc7\x02v\x83\xdc\xabt\x7f\xde\xb1\x9c\x04x\xc3\xffc{}\x82 \x1d\x9eu\xd72(\xd2\x92\xd7\xec\xa1O\xb9\x995\x96%s\x03_\x14+^G|\xed.\x8a\xc4`t\x9d;E\xee,K\xafT5\x96l\x00\xf5\x18\xaa\nk\xba\xf1\xfa\\s\x1dy\xa2z\xb6o$V\x85\x89\x03\xfd\x05\x9a@\xe0\x14\xb6Vu\tP5\xef\xbe3m\xb5m\xec\xf2C\x97\xe4\xcb^\x95\x88<\xeb\xa4"z\xd1\xbfS(\x93\x0e\xedo\xd6\xab(\xab\xe8`\x8e\x92.\xce\x1a\xfat\x19\xf23\xf9\xcf\xb5\xc1\x9b\xa0\xb9/\xea\xca\x84\xb32\xab.U\xf7\xaa\x97>\xab/\xb8\xbc#\xa0\xe3)\x1e%\xdb\xf29\xe2W\xf6\xe7\x17\xd8\x04\xd7\x11\x0f\xdf)>\x9b=\xea\xfd\x18\xf8\xf6\xf1\xde3 \x98=Hj1\x90\xb6\x12\x1e0\xc2(\xf5\xa3\x9e\x10\xd4e\x81cm\xe1\x1d\xb4\x01\xea\x9c\x7f\x08\x1d\x1e\x1f;#D}\xce\xa5EY\xd1\xa6H\xfe\xc0\xe3\xb7\x06\xe6BQ\x0b\xed}\x1d\xad'
Cipher Text: b'pI\x85,\x98-\xc4\xec\x98Y\x1e\xce\x08\x90RO'
AES_IV：b'P+"\x12f\xf45\xb3\xa9\xa5v\xc4\xe0#~\xed'


In [45]:
# 导入RSA私钥
key = RSA.import_key(open('data/rsa_key.pem').read())

# Decrypt the AES key with the private RSA key
cipher_rsa = PKCS1_OAEP.new(key)
dec_aes_key = cipher_rsa.decrypt(enc_aes_key)
print('Decrypted AES Key: {}'.format(dec_aes_key))

# Decrypt the data with the AES key
cipher_aes = AES.new(dec_aes_key, AES.MODE_CBC, iv=aes_iv)
plain_text = unpad(cipher_aes.decrypt(cipher_text), AES.block_size)
print('Plain Text: {}'.format(plain_text))

Decrypted AES Key: b'A[\x9d\xd7?\x11\x93\xa1\xbb\x86\xa0\x16\xc4\x1c\xf1x'
Plain Text: b'hello world.'
